# Data Modelling

## Get the data from the previous notebook

##### Import the libraries

In [ ]:
import json
import psycopg2
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [ ]:
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

##### Get the data from the previous notebook, as csv files, and load them into dataframes

In [ ]:
awards_players_df = pd.read_csv('../prep_data/dfs/awards_players_df.csv')
coaches_df = pd.read_csv('../prep_data/dfs/coaches_df.csv')
players_df = pd.read_csv('../prep_data/dfs/players_df.csv')
players_teams_df = pd.read_csv('../prep_data/dfs/players_teams_df.csv')
series_post_df = pd.read_csv('../prep_data/dfs/series_post_df.csv')
teams_post_df = pd.read_csv('../prep_data/dfs/teams_post_df.csv')
teams_df = pd.read_csv('../prep_data/prepared_dataset.csv')

# Make a dictionary with all the dataframes
dfs = {'awards_players_df': awards_players_df, 'coaches_df': coaches_df, 'players_df': players_df, 'players_teams_df': players_teams_df, 'series_post_df': series_post_df, 'teams_df': teams_df, 'teams_post_df': teams_post_df}

### Get the data from competition year

In [ ]:
# DB Credentials
with open("../config.json") as config_file:
    config = json.load(config_file)

host = config["db_host"]
user = config["db_user"]
password = config["db_password"]
database = config["db_database"]
schema = config["db_11_schema"]

In [ ]:
connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor()

def execute(query):
    cursor.execute(query)
    connection.commit()
    return cursor.fetchall()

def fetch(query):
    cursor.execute(query)
    return cursor.fetchall()

SELECT = "SELECT * FROM " + schema + "." # + table_name 
INSERT = "INSERT INTO " + schema + "." # + table_name + " VALUES " + values
UPDATE = "UPDATE " + schema + "." # + table_name + " SET " + column_name + " = " + value
DELETE = "DELETE FROM " + schema + "."  # + table_name + " WHERE " + column_name + " = " + value

In [ ]:
coaches = fetch(SELECT + "coaches") # all coaches who've managed the teams during the time period,
players_teams = fetch(SELECT + "players_teams") # performance of each player for each team they played,
teams = fetch(SELECT + "teams") # performance of the teams for each season,

players_teams_11_df = pd.DataFrame(players_teams, columns=['playerID', 'year', 'stint', 'tmID', 'lgID'])
teams_11_df = pd.DataFrame(teams, columns=['year', 'lgID', 'tmID', 'franchID', 'confID', 'name', 'arena', 'playoff'])
coaches_11_df = pd.DataFrame(coaches, columns=['coachID', 'year', 'tmID', 'lgID', 'stint'])

##### Make some preprocessing to year 11 data to make it compatible with the other years

In [ ]:
# Iterate over each row in the DataFrame
def franchise_mapping(teams_df, dfs):
    team_franchise_mapping = {}
    franchise_team_mapping = {}  # dictionary for reverse mapping
    
    for _, row in teams_df.iterrows():
        # Extract team and franchise IDs from the current row
        team_id = row['tmID']
        franchise_id = row['franchID']

        # Check if the team ID is not already in the mapping dictionary
        if team_id not in team_franchise_mapping:
            # Add the team ID and its corresponding franchise ID to the mapping
            team_franchise_mapping[team_id] = franchise_id
            franchise_team_mapping[franchise_id] = team_id

    # Now, team_franchise_mapping contains the mapping between team IDs and franchise IDs
    print(team_franchise_mapping)

    for _, df in dfs.items():
        # Check if 'tmID' is a column in the current DataFrame
        if 'tmID' in df.columns:
            # Replace team IDs with franchise IDs using the mapping
            df['tmID'] = df['tmID'].map(team_franchise_mapping)
        if 'franchID' in df.columns:
            # Drop the 'franchID' column
            df.drop(columns=['franchID'], inplace=True)

    return team_franchise_mapping, franchise_team_mapping

In [ ]:
binary_columns = ['confID', 'playoff']

for col in binary_columns:
    teams_11_df[col] = teams_11_df[col].replace('EA', 0)
    teams_11_df[col] = teams_11_df[col].replace('WE', 1)
    teams_11_df[col] = teams_11_df[col].replace('N', 0)
    teams_11_df[col] = teams_11_df[col].replace('Y',1)

team_franchise_map, franchise_team_map = franchise_mapping(teams_11_df, {'teams': teams_11_df, 'players_teams': players_teams_11_df, 'coaches':coaches_11_df})


players_teams_11_df = players_teams_11_df.reindex(columns=players_teams_df.columns)
players_teams_df = pd.concat([players_teams_11_df, players_teams_df])
dfs['players_teams_df'] = players_teams_df

teams_11_df = teams_11_df.reindex(columns=teams_df.columns)
teams_df = pd.concat([teams_11_df, teams_df])
dfs['teams_df'] = teams_df

coaches_11_df = coaches_11_df.reindex(columns=coaches_df.columns)
coaches_df = pd.concat([coaches_11_df, coaches_df])
dfs['coaches_df'] = coaches_df

### Normalization

We will normalize the data, so models likes knn, linear regression and logistic regression, that assume that the data is normally distributed, can be used.

In [ ]:
# for the numeric columns in players_teams_df verify if they follow gaussian distribution
def gaussian_distribution(df, output = False):
    gaussian_columns = []
    non_gaussian_columns = []

    for col in df.select_dtypes(include=np.number).columns:
        # Check if the column has 'NA' values
        if 'NA' in df[col].unique():
            continue

        # Evaluate if the column follows a Gaussian distribution
        _, p_value = stats.shapiro(df[col].dropna())  # Drop 'NA' values for the test
        alpha = 0.05
        if p_value > alpha:
            gaussian_columns.append(col)
            if output:
                print('{} follows a Gaussian distribution'.format(col))
        else:
            non_gaussian_columns.append(col)
            if output:
                print('{} does not follow a Gaussian distribution'.format(col))

    return gaussian_columns, non_gaussian_columns

gaussian_distribution(dfs['teams_df'], True)

In [ ]:
def normalize_df(X_train, X_test):
    X_train_normalized = X_train.copy()
    X_test_normalized = X_test.copy()
    
    gaussian_columns, non_gaussian_columns = gaussian_distribution(X_train_normalized)
    
    # For the columns that follow a Gaussian distribution, apply StandardScaler; otherwise, apply MinMaxScaler
    for col in non_gaussian_columns:
        if col not in ['year', 'confID', 'playoff']:
            scaler = MinMaxScaler()
            X_train_normalized[col] = scaler.fit_transform(X_train_normalized[[col]])
            X_test_normalized[col] = scaler.transform(X_test_normalized[[col]])

    for col in gaussian_columns:
        if col not in ['year', 'confID', 'playoff']:
            scaler = StandardScaler()
            X_train_normalized[col] = scaler.fit_transform(X_train_normalized[[col]])
            X_test_normalized[col] = scaler.transform(X_test_normalized[[col]])
    
    return X_train_normalized, X_test_normalized

# MODELING

### Constants and functions

Define some constants that we will be going to use across the notebook

In [ ]:
TARGET_NAMES = ['No Playoffs', 'Playoffs']
N_FOLDS_CV = 10
PREDICTION_YEAR = 10 # Will be changed to 11 later

##### Create reusable functions that will be useful across the notebook

- `print_confusion_matrix`:
Prints a confusion matrix as a heatmap, based on the provided true and predicted values.

- `print_results`:
Fits the model, predicts values, and outputs both the confusion matrix and accuracy score.

- `cross_validation`:
Conducts cross-validation on the model using the provided data.

- `force_qualify_8_teams`:
As we know from the WNBA rules, there are 8 teams that qualify for the playoffs. This function will be used to force the qualification of 8 tams, 4 from each conference, based on the predicted probabilities and true values.

- `plot_learning_curve`:
Generates a learning curve plot for a given model.


In [ ]:
def print_confusion_matrix(cm, target_names):
    print(cm)
    df_cm = pd.DataFrame(cm, index=target_names, columns=target_names)
    sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def print_results(clf, X_train, X_test, y_train, y_test, refit=True, output=True):
    # Fit the classifier on the training data
    if refit:
        clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Calculate the accuracy of the classifier
    if output:
        accuracy = accuracy_score(y_test, y_pred)
        print("Accuracy:", accuracy)

    # Create a confusion matrix to evaluate the model and print with labels
    if output:
        cm = confusion_matrix(y_test, y_pred)
        print_confusion_matrix(cm, TARGET_NAMES)

    return y_pred

def cross_validation(clf, X, y, output=True):
    cv = StratifiedKFold(n_splits=N_FOLDS_CV, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Display the cross-validation results
    if output:
        print("Cross-Validation Results:")
        print("Mean Accuracy: {:.2f}%".format(scores.mean() * 100))
        print("Standard Deviation: {:.2f}".format(scores.std()))

    return scores
    
def force_qualify_8_teams(clf, X_test, y_test, output=True):
    y_pred_probs = clf.predict_proba(X_test)[:, 1]  # Get the probabilities for each team

    # Combine predicted probabilities with conference information
    pred_df = pd.DataFrame({'Probability': y_pred_probs, 'ConfID': X_test['confID']})

    # Select the top 4 teams from each conference based on predicted probabilities
    top_teams_indices = pred_df.groupby('ConfID')['Probability'].nlargest(4).index.get_level_values(1)

    # Convert the labels to numeric format (0 for 'No Playoff', 1 for 'Playoff')
    y_pred_numeric = [1 if i in top_teams_indices else 0 for i in y_test.index]

    # Calculate the accuracy of the modified predictions
    accuracy = accuracy_score(y_test, y_pred_numeric)
    if output:
        print("Modified Accuracy (Top 4 Teams from Each Conference as Playoff):", accuracy)

    # Create a confusion matrix to evaluate the model and print with labels
    if output:
        cm = confusion_matrix(y_test, y_pred_numeric)
        print_confusion_matrix(cm, TARGET_NAMES)

    return y_pred_numeric

def plot_learning_curve(clf, title, X, y, ylim=None, cv=10, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Loss") 
    train_sizes, train_losses, test_losses = learning_curve(
        clf, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='neg_mean_squared_error')  # Use neg_mean_squared_error as a loss
    train_losses_mean = -np.mean(train_losses, axis=1)  # Reverse the sign for loss
    train_losses_std = np.std(train_losses, axis=1)
    test_losses_mean = -np.mean(test_losses, axis=1)  # Reverse the sign for loss
    test_losses_std = np.std(test_losses, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_losses_mean - train_losses_std,
                     train_losses_mean + train_losses_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_losses_mean - test_losses_std,
                     test_losses_mean + test_losses_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_losses_mean, 'o-', color="r",
             label="Training loss")
    plt.plot(train_sizes, test_losses_mean, 'o-', color="g",
             label="Validation loss")

    plt.legend(loc="best")
    return plt

## With current year info prediction

Run a simple model on the teams table to predict if a team will make it to the playoffs or not. Let's see how it behaves.

The main objective of this chapter is to find which is the model that performs the best, given the perfect data (the data from the year to be predicted).

We will only run simple models with the data of current year, because we expected them to perform very well as they are trained with the data that already tells us which teams will make it to the playoffs.

In [ ]:
# Split data into train years and test year (year 10)
teams_train_df = teams_df[teams_df['year'] < PREDICTION_YEAR]
teams_test_df = teams_df[teams_df['year'] == PREDICTION_YEAR]

cross_val_scores = []

In [ ]:
y_train = teams_train_df['playoff']
y_test = teams_test_df['playoff']
X_train = teams_train_df.drop(['playoff', 'year', 'tmID'], axis=1)
X_test = teams_test_df.drop(['playoff', 'year', 'tmID'], axis=1)

# Normalize the data as KNN, Neural Networks, SVM, ... expect the data to be normalized
X_train_normalized, X_test_normalized = normalize_df(X_train, X_test)

X_normalized = pd.concat([X_train_normalized, X_test_normalized])
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

### Random Forest

In [ ]:
clf = RandomForestClassifier()

y_pred = print_results(clf, X_train, X_test, y_train, y_test)

plot_learning_curve(clf, 'Random Forest Learning Curve', X, y)

As we expected, the model performs very well, with a 100% accuracy. From the learning curve we can see that the model is a good model, as the training and validation losses are low and are very close to each other.

In [ ]:
force_qualify_8_teams(clf, X_test, y_test)
scores = cross_validation(clf, X, y)
cross_val_scores.append(scores)

### KNN

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)

y_pred = print_results(knn_classifier, X_train_normalized, X_test_normalized, y_train, y_test)

y_pred = force_qualify_8_teams(knn_classifier, X_test_normalized, y_test)

scores = cross_validation(knn_classifier, X_normalized, y)
cross_val_scores.append(scores)

plot_learning_curve(knn_classifier, 'KNN Learning Curve', X_normalized, y)

KNN didn't perform as well as Random Forest, but it still performed very well, with a 91% accuracy. From the learning curve we can see that the model is underfitting, as the losses are quite significant and the training and validation losses are far from each other.

### Conclusions

Using the current year information, the best model at their best. This permitted as to see which models will have a better performance with the data from the previous years, when we don't have the current year information.

In [ ]:
plt.boxplot(cross_val_scores, labels=['Random Forest', 'KNN'])
plt.title('Cross-Validation Scores')
plt.ylabel('Accuracy')
plt.show()

The Random Forest Classifier seems to be performing better than the other classifiers.

In the next chapter we will explore using a rolling window to use past years for the predictions

## Rolling window

### Rolling window functions

We will use a rolling window to predict the next season.
In this dataset, 3 years are combined, with more weight given to the most recent year, and combining that historic data with the target variable (playoff) from the target year (the n+1 year)

In [ ]:
# Define the rolling window size (number of past years to consider)
ROLLING_WIN_SIZE = 3  # You can adjust this value
PREDICTION_YEAR = 11

def create_rolling_window_dataset(df, rolling_win_size):
    '''This function creates a list of dataframes, where each 
    dataframe contains the data from the previous n years.
    It creates an entry for each year that the team has played.'''

    df_rolling = []
    for year in range(rolling_win_size + 1, PREDICTION_YEAR + 1):
        df_year = df[df['year'] == year]
        df_prev_years = df[(df['year'] < year) & (df['year'] >= year - rolling_win_size)]

        # Take the data into a df for each team
        for team in df_year['tmID'].unique():
            # Take the data from the previous years for the same team
            df_team_prev_years = df_prev_years[df_prev_years['tmID'] == team]

            # Add the dataframe to the list
            df_rolling.append(df_team_prev_years)

    return df_rolling

def generate_weights(window_size):
    weights = np.linspace(0.1, 1, window_size)
    weights /= weights.sum()  # Normalize weights to ensure they sum to 1
    return weights

In [ ]:
def rolling_window_df():
    # Create the rolling window datasets
    rolling_window_historic = create_rolling_window_dataset(teams_df,  ROLLING_WIN_SIZE)

    weighted_features = teams_df.drop(['year', 'tmID', 'confID', 'playoff'], axis=1).select_dtypes(include=np.number).columns.tolist()
    
    # Create a new dataframe to store the rolling window data
    rolling_data = pd.DataFrame(columns=teams_df.columns)

    for window in rolling_window_historic:

        #Create a new empty line for the rolling_data dataframe
        rolling_data_line = pd.DataFrame(columns=teams_df.columns)

        if len(window) != 0:
            #1. Get the weights for the weighted average
            weights = generate_weights(len(window))

            #2. Get the weighted average of the weighted features
            #Note: The weighted average is calculated as follows:
            #weighted average = sum of (weight * feature value) / sum of weights
            #For example, if the weights are [0.2, 0.3, 0.5] and the feature values are [10, 20, 30], then the weighted average is:
            #weighted average = (0.2 * 10 + 0.3 * 20 + 0.5 * 30) / (0.2 + 0.3 + 0.5) = 24
            for feature in weighted_features:
                weighted_average = np.average(window[feature].values, weights=weights)
                rolling_data_line[feature] = [weighted_average]

            #3. Join that data with the data from the following year (the year we want to predict)
            #Note: The data from the following year will be used as the target label, get next year from the teams_df dataframe
            next_year = window['year'].max() + 1
            next_year_data = teams_df[(teams_df['tmID'] == window['tmID'].values[0]) & (teams_df['year'] == next_year)]

        #Add year, confID, tmID, and playoff columns
        try:
            rolling_data_line['year'] = next_year
            rolling_data_line['tmID'] = window['tmID'].values[0]
            rolling_data_line['confID'] = next_year_data['confID'].values[0]
            rolling_data_line['playoff'] = next_year_data['playoff'].values[0]
        except:
            #If there is no data for the next year, skip this line
            #print('No data for next year')
            continue
        
        #4. Concat the data to the rolling_data dataframe
        rolling_data = pd.concat([rolling_data, rolling_data_line])

    rolling_data = rolling_data.reset_index(drop=True)

    return rolling_data

In [ ]:
rolling_data = rolling_window_df()
rolling_data.head()

### Random Forest with Rolling window

In [ ]:
y_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR]['playoff'].astype(int)
y_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['playoff'].astype(int)
X_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR].drop(['playoff', 'year', 'tmID'], axis=1).dropna(axis=1)
X_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR].drop(['playoff', 'year', 'tmID'], axis=1).dropna(axis=1)
X_train_normalized, X_test_normalized = normalize_df(X_train, X_test)

X = pd.concat([X_train, X_test])
X_normalized = pd.concat([X_train_normalized, X_test_normalized])
y = pd.concat([y_train, y_test])

In [ ]:
clf = RandomForestClassifier(n_estimators=100)

y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(clf, X_test, y_test)

scores = cross_validation(clf, X, y)

plot_learning_curve(clf, 'Random Forest Learning Curve', X, y)

The learning curve tells us that the model with 100 estimators does not look like a good model, as the training and validation losses are far from each other and the losses are quite significant.

## Grid Search

Now we will perform some grid search on some models with the rolling window data, to see if we can improve the results.

### Random Forest

Now, we will try to find what are the best parameters for the Random Forest, KNN and more models, using Grid Search, and tune this models to see if we can improve the results.

In [ ]:
# Use grid search to find the best parameters
cross_val_scores = []
best_clf = {}
grid_search_params = {}
grid_search_models = {}

parameters = {
    'n_estimators': [5, 10, 50],
    'max_depth': [3, 5, 11], 
    'max_features': ['sqrt', 'log2', None], 
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10] 
}

clf = RandomForestClassifier()
grid_search_models['Random Forest'] = clf
grid_search_params['Random Forest'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

clf = grid_search.best_estimator_
best_clf['Random Forest'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(clf, X_test, y_test)
scores = grid_search.cv_results_['mean_test_score']
plot_learning_curve(clf, 'Random Forest best estimator Learning Curve', X, y)
cross_val_scores.append(scores)

The curve is showing that the model improved a little bit, as the validation losses are closer now to the training losses, but the losses are still quite significant.

### KNN

In [ ]:
# Use grid search to find the best parameters
parameters = {'n_neighbors': [3, 5, 7, 9, 11, 13], 'weights': ['uniform', 'distance']}
clf = KNeighborsClassifier()
grid_search_models['KNN'] = clf
grid_search_params['KNN'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_normalized, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

clf = grid_search.best_estimator_
best_clf['KNN'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train_normalized, X_test_normalized, y_train, y_test)
force_qualify_8_teams(clf, X_test_normalized, y_test)
scores = grid_search.cv_results_['mean_test_score']
plot_learning_curve(clf, 'KNN best estimator Learning Curve', X_normalized, y)
cross_val_scores.append(scores)

On this model, the learning curve is having too high values of loss, even though the training and validation losses are close to each other. 

### Gradient Boosting

In [ ]:
parameters = {
    "learning_rate": [0.01, 0.1],
    "min_samples_split": np.linspace(0.1, 0.5, 2),
    "min_samples_leaf": np.linspace(0.1, 0.5, 2),
    "max_depth":[3,8],
    "max_features":["log2",None],
    "subsample":[0.8, 1.0],
    "n_estimators":[100]
}
clf = GradientBoostingClassifier()
grid_search_models['Gradient Boosting'] = clf
grid_search_params['Gradient Boosting'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

clf = grid_search.best_estimator_
best_clf['Gradient Boosting'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(clf, X_test, y_test)
scores = grid_search.cv_results_['mean_test_score']
plot_learning_curve(clf, 'Gradient Boosting best estimator Learning Curve', X, y)
cross_val_scores.append(scores)

Gradient Boost can be better than Random Forest, but it is more prone to overfitting and it is more difficult to tune. At this moment Random Forest is performing better than Gradient Boost.

### XBoost

In [ ]:
X_train['confID'] = X_train['confID'].astype('bool')
X_test['confID'] = X_test['confID'].astype('bool')
X = pd.concat([X_train, X_test])

parameters = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.8, 1.0],
    'n_estimators': [100, 200]
}
clf = xgb.XGBClassifier()
grid_search_models['XGBoost'] = clf
grid_search_params['XGBoost'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

clf = grid_search.best_estimator_
best_clf['XGBoost'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(clf, X_test, y_test)
scores = grid_search.cv_results_['mean_test_score']
cross_val_scores.append(scores)
plot_learning_curve(clf, 'XGBoost best estimator Learning Curve', X, y)

### SVM

In [ ]:
parameters = {
    'C': [0.01, 0.1, 1], 
    'gamma': ['scale'], 
    'kernel': ['linear'],
    'class_weight': ['balanced'],
    'probability': [True]
}
clf = svm.SVC()
grid_search_models['SVM'] = clf
grid_search_params['SVM'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_normalized, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
clf = grid_search.best_estimator_
best_clf['SVM'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train_normalized, X_test_normalized, y_train, y_test)
force_qualify_8_teams(clf, X_test_normalized, y_test)
scores = grid_search.cv_results_['mean_test_score']
plot_learning_curve(clf, 'SVM best estimator Learning Curve', X_normalized, y)
cross_val_scores.append(scores)

### Logistic Regression

In [ ]:
parameters = {
    'C': [0.01, 0.1, 1],
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced'],
    'solver': ['liblinear'],
    'max_iter': [100],
}

clf = LogisticRegression()
grid_search_models['LogisticRegression'] = clf
grid_search_params['LogisticRegression'] = parameters

grid_search = GridSearchCV(clf, parameters, cv=10, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train_normalized, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
print("Best Estimator:", grid_search.best_estimator_)

clf = grid_search.best_estimator_
best_clf['LogisticRegression'] = (grid_search.best_score_, clf)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train_normalized, X_test_normalized, y_train, y_test)
force_qualify_8_teams(clf, X_test_normalized, y_test)
scores = grid_search.cv_results_['mean_test_score']
plot_learning_curve(clf, 'Logistic Regression Learning Curve', X_normalized, y)
cross_val_scores.append(scores)

### Plotting the results

In [ ]:
#plot the cross validation scores
plt.boxplot(cross_val_scores, labels=best_clf.keys())
plt.title('Cross-Validation Scores')
plt.ylabel('Accuracy') 
plt.show()

After getting our best classifier, we will try different numbers of ROLLING WINDOW to see if we can improve the results.

In [ ]:
# Initialize variables to store the best score and estimator
best_score = -1
best_estimator = None
best_clf_name = None

# Iterate over the best_clf dictionary
for clf_name, (score, estimator) in best_clf.items():
    # If this classifier's score is higher than the current best score, update the best score and estimator
    if score > best_score:
        best_score = score
        best_estimator = estimator
        best_clf_name = clf_name

print(f"The best classifier is {best_clf_name} with a score of {best_score}")

cross_val_scores = []
best_year = 0
best_score = -1
for i in range(1, 10):
    ROLLING_WIN_SIZE = i
    rolling_data = rolling_window_df()

    clf = best_estimator

    X = pd.concat([X_train, X_test])
    y = pd.concat([y_train, y_test])

    y_pred = print_results(clf, X_train, X_test, y_train, y_test, output=False)

    scores = cross_validation(clf, X, y, False)
    cross_val_scores.append(scores)

    if scores.mean() > best_score:
        best_score = scores.mean()
        best_year = i

plt.boxplot(cross_val_scores, labels=['1', '2', '3', '4', '5', '6', '7', '8', '9'])

Now we will use the rolling window we created on the previous notebook, we will base our statistics on the last years of the players that currrently play on the roster of the team.
As we could ee from before, when creating the rolling window data the more years we use in the past, less their weight. So we can barely notice any difference between using 3 or 10 years of data into account. So we will use 3 years of data, as it is the most recent data for this next part

In [ ]:
labels = []
cross_val_scores = []

rolling_data = pd.read_csv('../prep_data/rolling_window/data_with_' + str(3) +'_years_in_the_past.csv')

y_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR]['playoff'].astype(int)
y_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['playoff'].astype(int)
confIDs = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['confID']
tmIDs = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['tmID']
X_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR].drop(['playoff', 'year', 'tmID'], axis=1).dropna(axis=1)
X_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR].drop(['playoff', 'year', 'tmID'], axis=1).dropna(axis=1)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

for model in grid_search_models.keys():
    if model in ['KNN', 'SVM', 'XGBoost']:
        # Normalize the data as KNN, SVM, ... expect the data to be normalized
        X_train_model, X_test_model = normalize_df(X_train, X_test)
        X_model = pd.concat([X_train_model, X_test_model])
    else:
        X_train_model, X_test_model = X_train, X_test
        X_model = X
        
    clf = grid_search_models[model]
    parameters = grid_search_params[model]

    labels.append(model + '_w' + str(3) + 'years')
    
    clf = best_clf[model][1]
    clf.fit(X_train_model, y_train)
    

    best_clf[model] = (best_clf[model][0], best_clf[model][1], [X_train_model, X_test_model, y_train, y_test], 3)

    y_pred = print_results(clf, X_train_model, X_test_model, y_train, y_test, output=False)

    scores = cross_validation(clf, X_model, y, False)
    cross_val_scores.append(scores)

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.boxplot(cross_val_scores, labels=labels)
plt.xticks(rotation=45, ha="right")
plt.title('Rolling data size: ' +  str(3))
plt.show()

best_score = -1
best_classifier = None
for best_name, best in best_clf.items():
    print(best_name + '( score: ' + str(best[0]) + ', with ' + str(3) + ' years behind, estimator:' + str(best[1]) + ')')

    if best[0] > best_score:
        best_score = best[0]
        best_classifier = best

In [ ]:
competition_data = pd.read_csv('../prep_data/rolling_window/data_with_' + str(3) +'_years_in_the_past.csv')

with open('results.txt', 'w') as f:
    y_pred_without_8 = print_results(best_classifier[1], best_classifier[2][0], best_classifier[2][1], best_classifier[2][2], best_classifier[2][3])
    y_pred = force_qualify_8_teams(best_classifier[1], best_classifier[2][1], y_test)

    X = pd.concat([best_classifier[2][0], best_classifier[2][1]])
    y = pd.concat([best_classifier[2][2], best_classifier[2][3]])
    # Convert y_pred_numeric to a DataFrame and reset the index
    y_pred_df = pd.DataFrame(y_pred, columns=["playoff_predict"])
    y_pred_without_8_df = pd.DataFrame(y_pred_without_8, columns=["playoff_predict_without_8"])

    # Filter competition_data for the prediction year
    competition_data = competition_data[rolling_data['year'] == PREDICTION_YEAR][['tmID', 'year', 'confID', 'playoff']].sort_values('tmID', ascending=True)

    # Merge competition_data with y_pred_df
    result = competition_data.reset_index(drop=True).join(y_pred_df)
    result = result.reset_index(drop=True).join(y_pred_without_8_df).sort_values(['confID', 'tmID'], ascending=True)

    result['playoff'] = result['playoff'].map({1: 'Y', 0: 'N'})
    result['tmID'] = result['tmID'].map(franchise_team_map)

    result_subset = result[['tmID', 'playoff', 'playoff_predict', 'playoff_predict_without_8']]
    result_subset = result_subset.sort_values('tmID')

    # Write the result to a CSV file
    result_subset.to_csv('output_data/output.csv', index=False)

In [ ]:
# import pickle
# with open('output_data/best.pkl', 'wb') as model_file:
#     pickle.dump(best_classifier[1], model_file)